In [1]:
import os

import dolfinx
import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pyvista
import pyvista as pv
import pyvistaqt as pvqt
import ufl
import warnings

from dolfinx import cpp, default_real_type, default_scalar_type, fem, io, la, mesh, nls, plot
from dolfinx.fem import petsc
from dolfinx.io import gmshio, VTXWriter
from dolfinx.nls import petsc as petsc_nls
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells
from IPython.display import Image

from mpi4py import MPI
from petsc4py import PETSc
from ufl import (Circumradius, FacetNormal, SpatialCoordinate, TrialFunction, TestFunction,
                 dot, div, dx, ds, dS, grad, inner, grad, avg, jump)

import utils

In [2]:
class Markers:
    def __init__(self):
        pass

    @property
    def left(self):
        return 1
    
    @property
    def right(self):
        return 2
    
    @property
    def domain(self):
        return 1

workdir = "output/poisson_nernst_planck"
utils.make_dir_if_missing(workdir)
output_meshfile = os.path.join(workdir, "mesh.msh")
output_file = os.path.join(workdir, "u.bp")
# c_p_file = os.path.join(workdir, "c_p.bp")
# phi_file = os.path.join(workdir, "phi.bp")
point_0 = (0, 0, 0)
point_1 = (150e-6, 0, 0)
markers = Markers()
gmsh.initialize()
gmsh.model.add('interval')
p0 = gmsh.model.occ.addPoint(*point_0, meshSize=1e-6)
p1 = gmsh.model.occ.addPoint(*point_1, meshSize=1e-6)
interval = gmsh.model.occ.addLine(p0, p1)
gmsh.model.occ.synchronize()
gmsh.model.addPhysicalGroup(0, [p0], markers.left, "left")
gmsh.model.addPhysicalGroup(0, [p1], markers.right, "right")
gmsh.model.addPhysicalGroup(1, [interval], markers.domain, "interval")
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(1)
gmsh.write(output_meshfile)
gmsh.finalize()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 0.000128454s, CPU 0.000113s)
Info    : 151 nodes 152 elements
Info    : Writing 'output/poisson_nernst_planck/mesh.msh'...
Info    : Done writing 'output/poisson_nernst_planck/mesh.msh'


In [3]:
D_n = 1e-10
D_p = 1e-10
F = 96485
R = 8.3145
T = 298
m_n = D_n / (R * T)  # mobility of negative
m_p = D_p / (R * T)  # mobility of positive
z_n = -1
z_p = 1
dt = 1e-6

In [4]:
comm = MPI.COMM_WORLD
partitioner = mesh.create_cell_partitioner(mesh.GhostMode.shared_facet)
domain, ct, ft = gmshio.read_from_msh(output_meshfile, comm, partitioner=partitioner)
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(tdim, fdim)

Info    : Reading 'output/poisson_nernst_planck/mesh.msh'...
Info    : 3 entities
Info    : 151 nodes
Info    : 152 elements
Info    : Done reading 'output/poisson_nernst_planck/mesh.msh'


In [5]:
V = dolfinx.fem.functionspace(domain, ("CG", 1, (3,), ))
u0 = dolfinx.fem.Function(V) # previous step.
u = dolfinx.fem.Function(V) # current step.
c0_n, c0_p, phi0 = ufl.split(u0)
c_n, c_p, phi = ufl.split(u)
v_n, v_p, v_phi = ufl.TestFunction(V)
n = FacetNormal(domain)
dx = ufl.Measure("dx", domain=domain, subdomain_data=ct)#, metadata={"quadrature_degree": 4})
ds = ufl.Measure("ds", domain=domain, subdomain_data=ft)#, metadata={"quadrature_degree": 4})

In [6]:
# initial concentrations
Vcell = 1e-3
u0.sub(0).interpolate(lambda x: 1e4 + x[0] - x[0])
u0.sub(1).interpolate(lambda x: 1e4 + x[0] - x[0])
# initial potential 
u0.sub(2).interpolate(lambda x: Vcell * x[0] / 150e-6)
u0.x.scatter_forward()

## Poisson-Nernst-Planck Equation
The flux of species $i$ is given by the Nernst-Planck equation
$$\pmb{N}_i = -z_i u_i F \nabla \phi - D_i \nabla c_i + c_i \pmb{v}$$
Material balance is given by
$$\dfrac{\partial c_i}{\partial t} = -\nabla \cdot \pmb{N}_i + \mathcal{R}_i$$
where $\mathcal{R}_i$ is the homogeneous reaction.

The relationship between diffusivity $D_i$ and mobility $u_i$ is given by the Nernst-Einstein relationship
$$D_i = R T u_i$$

To solve the Nernst-Planck equation for two charged species, we need three test functions and three trial functions---one for +ve species, one for -ve species and one for potential.

An additional equation that has to be included is the charge conservation equation in the form of Poisson equation, to give the pair of equations called the Poisson-Nernst-Planck equation.
$$\nabla \cdot (-\kappa\nabla\phi)=0$$

In battery systems, one can ignore the advection term $c_i\pmb{v}$ if the system is not driven by bulk flow. Such battery systems include Li-ion batteries.
### Boundary and Initial Conditions
- initial concentration of both +ve and -ve species must be given
- initial potential gradient must be provided
- normal flux of -ve species is zero on the external boundary $\partial \Omega$
- normal flux of +ve species, assumed to be Li$^+$, is zero on insulated boundary and non-zero on the boundaries where charge transfer occurs at
- normal gradient of potential is zero on the insulated boundaries
- normal gradient of potential and normal flux of Li$^+$ are coupled at the charge transfer boundaries

### Variational Formulation
For each charged species conservation equation, multiply the Nernst-Planck equation by the test function $v_i$ and integrate over the domain $\Omega$
\begin{equation}
\int_{\Omega}\dfrac{\partial c_i}{\partial t}v_i\mathrm{dx} = -\int_{\Omega}\nabla \cdot \pmb{N}_i v_i \mathrm{dx}
\end{equation}

After integrating by parts and using backward Euler time-integration, we obtain
\begin{equation}
\int_{\Omega}(c_i|_{t+1}-c_i|_{t})v_i\mathrm{dx} = \mathrm{dt}\Bigl[ \int_{\Omega}\nabla v_i \cdot \pmb{N}_i \mathrm{dx} - \int_{\partial\Omega}\pmb{N}_iv_i\cdot\pmb{n}\mathrm{ds}\Bigr]
\end{equation}

For the Poisson equation, we have:
\begin{equation}
\int_{\Omega}\nabla\cdot(-\kappa\nabla\phi)v_{\phi}\mathrm{dx} = \int_{\Omega}\kappa\nabla\phi\cdot\nabla v_{\phi}\mathrm{dx} -\int_{\partial\Omega}\kappa\nabla\phi\cdot\pmb{n}v_{\phi}\mathrm{ds}
\end{equation}

Conductivity
$\kappa = F^2\sum_{i}z_i^2u_i c_i$

Transference number
$$
t_i \equiv \frac{z_i^2u_i c_i}{\sum_{j}z_j^2u_j c_j}
$$

In [7]:
left_boundary = ft.find(markers.left)
right_boundary = ft.find(markers.right)

In [8]:
zero = fem.Constant(domain, PETSc.ScalarType(0))
I_sup = fem.Constant(domain, PETSc.ScalarType(1))

In [9]:
u_left = fem.Function(V).sub(2)
with u_left.vector.localForm() as ul_loc:
    ul_loc.set(0)
left_bc = fem.dirichletbc(u_left, fem.locate_dofs_topological(V.sub(2), 0, left_boundary))

u_right = fem.Function(V).sub(2)
with u_right.vector.localForm() as ur_loc:
    ur_loc.set(Vcell)
right_bc = fem.dirichletbc(u_right, fem.locate_dofs_topological(V.sub(2), 0, right_boundary))

N_n = -z_n * m_n * F * c_n * grad(phi0) - D_n * grad(c0_n)
N_p = -z_p * m_p * F * c_p * grad(phi0) - D_p * grad(c0_p)
g_n = inner(-z_n * m_n * F * grad(phi), n)
obj_c_n = -inner(c_n, v_n) * dx + inner(c0_n, v_n) * dx + dt * inner(N_n, grad(v_n)) * dx - dt * inner(g_n, v_n) * ds(markers.left)  + dt * inner(g_n, v_n) * ds(markers.right)
obj_c_p = -inner(c_p, v_p) * dx + inner(c0_p, v_p) * dx + dt * inner(N_p, grad(v_p)) * dx #- dt * inner(-1e-4, v_p) * ds(markers.right)  #- dt * inner(1e-4, v_p) * ds(markers.right)
kappa = F ** 2 * (z_n ** 2 * m_n * c_n + z_p ** 2 * m_p * c_p)
obj_phi = kappa * inner(grad(phi), grad(v_phi)) * dx #+ zero * v_phi * ds(marker.)
obj = obj_c_n + obj_c_p + obj_phi

problem = petsc.NonlinearProblem(obj, u, bcs=[left_bc, right_bc])
solver = petsc_nls.NewtonSolver(comm, problem)
solver.convergence_criterion = "incremental"
# solver.maximum_iterations = 100
# solver.atol = np.finfo(float).eps
# solver.rtol = np.finfo(float).eps * 10

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}pc_type"] = "hypre"
ksp.setFromOptions()
TIME = 100 * dt
t = 0
L = fem.assemble_scalar(fem.form(1 * dx))
output_f = VTXWriter(comm, output_file, [u], engine="BP5")
output_f.write(0)

while t < TIME:
    t += dt
    n_iters, converged = solver.solve(u)
    c_n, c_p, phi = ufl.split(u)
    print(n_iters, converged)
    print(fem.assemble_scalar(fem.form(c_n * dx))/L, fem.assemble_scalar(fem.form(c_p * dx))/L)
    print(fem.assemble_scalar(fem.form(inner(-F * D_n * grad(c_n), n) * ds(markers.left))), fem.assemble_scalar(fem.form(inner(-F * D_n * grad(c_n), n) * ds(markers.right))))
    print(fem.assemble_scalar(fem.form(inner(-F * D_p * grad(c_p), n) * ds(markers.left))), fem.assemble_scalar(fem.form(inner(-F * D_p * grad(c_p), n) * ds(markers.right))))
    print(fem.assemble_scalar(fem.form(inner(-kappa * grad(phi), n) * ds(markers.left))), fem.assemble_scalar(fem.form(inner(-kappa * grad(phi), n) * ds(markers.right))))
    c0_n = c_n
    c0_p = c_p
    phi0 = phi
    output_f.write(t)
output_f.close()

3 True
10000.000000000357 10000.00000000001
0.01100081747728565 -0.011003019154825813
-0.011001919000525972 0.01100191763158549
50.09633301476914 -50.0963330149261
2 True
10000.000000000357 10000.00000000001
0.011000817442184611 -0.011003019172376333
-0.011001918982975454 0.011001917666686529
50.096333014770345 -50.09633301476966
2 True
10000.000000000357 10000.00000000001
0.011000817424634093 -0.011003019172376333
-0.011001919018076492 0.01100191764913601
50.09633301477035 -50.09633301476966
2 True
10000.000000000357 10000.00000000001
0.011000817424634093 -0.011003019172376333
-0.011001918982975454 0.01100191761403497
50.096333014770345 -50.09633301476967
2 True
10000.000000000357 10000.00000000001
0.011000817424634093 -0.011003019172376333
-0.011001918982975454 0.011001917666686529
50.09633301477035 -50.09633301476966
2 True
10000.000000000357 10000.00000000001
0.011000817424634093 -0.011003019172376333
-0.011001918982975454 0.01100191764913601
50.09633301477036 -50.09633301476966
2 